In [7]:
import cv2
import os
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from collections import deque
import math

In [3]:
mp_pose = mp.solutions.pose

def extract_landmarks(image, pose):
    """MediaPipe Pose를 사용하여 33개 관절 좌표를 추출"""
    results = pose.process(image)
    if not results.pose_landmarks:
        return None
    landmarks = []
    for lm in results.pose_landmarks.landmark:
        landmarks.extend([lm.x, lm.y, lm.z])
    return landmarks

def calculate_angle(a, b, c):
    """세 점 (a, b, c)으로 이루어진 각도를 계산"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

In [6]:
def process_videos(dataset_path, sequence_length=30):
    """dataset 폴더 내 모든 운동 폴더를 불러와 데이터를 생성하고 라벨링"""
    data = []
    labels = []
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
    
    label_map = {folder: i for i, folder in enumerate(os.listdir(dataset_path))}  # 폴더명을 라벨로 매핑
    print(f"📌 라벨 매핑: {label_map}")  # { 'pushup': 0, 'shoulder_press': 1, ... }
    
    for folder, label in label_map.items():
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):  
            continue  # 폴더가 아닌 파일이 있으면 무시
        
        print(f"🔍 {folder} 영상 처리 중... (라벨 {label})")
        
        for video_name in os.listdir(folder_path):
            video_path = os.path.join(folder_path, video_name)
            cap = cv2.VideoCapture(video_path)
            sequence = []

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                landmarks = extract_landmarks(frame_rgb, pose)

                if landmarks:
                    sequence.append(landmarks)
                    if len(sequence) == sequence_length:
                        data.append(sequence[:])
                        labels.append(label)
                        sequence.pop(0)  # 슬라이딩 윈도우 적용 (가장 오래된 프레임 삭제)

            cap.release()

    pose.close()
    return np.array(data), np.array(labels)


In [8]:
dataset_path = "/home/shin/deeplearning-repo-1/dataset"
X, y = process_videos(dataset_path)

# 라벨을 원-핫 인코딩
y = tf.keras.utils.to_categorical(y, num_classes=len(set(y)))

# 데이터셋 분할 (학습: 80%, 테스트: 20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ 데이터셋 크기: {X.shape}, 라벨 크기: {y.shape}")

I0000 00:00:1742454970.794570   98930 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742454970.796996   99235 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742454970.857296   99221 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742454970.886060   99219 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742454970.905199   99230 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


📌 라벨 매핑: {'standing': 0, 'pushup': 1, 'stand_knee_raise': 2, 'stand_shoulder_press': 3}
🔍 standing 영상 처리 중... (라벨 0)
🔍 pushup 영상 처리 중... (라벨 1)
🔍 stand_knee_raise 영상 처리 중... (라벨 2)
🔍 stand_shoulder_press 영상 처리 중... (라벨 3)
✅ 데이터셋 크기: (10769, 30, 99), 라벨 크기: (10769, 4)


In [9]:
# LSTM 모델 구성
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(30, 99)),  # 33개 관절 * 3(x,y,z)
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3개의 운동 분류
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습 (X_train, y_train 준비 필요)
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

/home/shin/venv/project/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
X_train.shape, y_train.shape

((8615, 30, 99), (8615, 4))

In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

2025-03-20 17:27:14.065731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742459234.156970    4757 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742459234.184317    4757 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742459234.390509    4757 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742459234.390531    4757 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742459234.390532    4757 computation_placer.cc:177] computation placer alr

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset'

SEQUENCE_LENGTH = 30

sequences = []
labels = []

for label_folder in os.listdir(DATASET_DIR):
    label_path = os.path.join(DATASET_DIR, label_folder)
    if not os.path.isdir(label_path):
        continue
    for video_file in os.listdir(label_path):
        video_path = os.path.join(label_path, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                landmarks = []
                for lm in results.pose_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
                frames.append(landmarks)
                if len(frames) >= SEQUENCE_LENGTH:
                    sequences.append(frames[-SEQUENCE_LENGTH:])
                    labels.append(label_folder)
        cap.release()

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

I0000 00:00:1742459248.771324    4757 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1742459248.774150    5509 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742459248.892202    5503 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742459248.938615    5504 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742459248.968797    5503 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [4]:
X = np.array(sequences)
y = np.array(labels_categorical)

X.shape, y.shape

((10769, 30, 132), (10769, 4))

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=labels_encoded)

In [7]:
early_stopping = EarlyStopping(
    monitor='loss',      # validation loss를 모니터링
    patience=10,         # 10 epoch 동안 개선 없으면 멈춤
    restore_best_weights=True # 가장 좋은 가중치 복원
)

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LENGTH, len(landmarks))))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(le.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

I0000 00:00:1742460053.278416    4757 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4227 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/shin/venv/project/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.fit(X, y, epochs=50, batch_size=100, callbacks=[early_stopping], validation_data=(X_val, y_val))

Epoch 1/50


I0000 00:00:1742460057.734958    7014 cuda_dnn.cc:529] Loaded cuDNN version 90800


108/108 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7693 - loss: 0.6215 - val_accuracy: 0.9865 - val_loss: 0.0719
Epoch 2/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9923 - loss: 0.0447 - val_accuracy: 1.0000 - val_loss: 5.6773e-04
Epoch 3/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss: 1.3082e-04
Epoch 4/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 5.2862e-05
Epoch 5/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 8.1940e-04 - val_accuracy: 1.0000 - val_loss: 2.6629e-05
Epoch 6/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 6.0172e-04 - val_accuracy: 1.0000 - val_loss: 1.4121e-05
Epoch 7/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 4.7631e-04 - val_accuracy: 1.0000 - val_loss: 9.0337e-06
Epoch 8/50
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 

In [9]:
model.save('exercise_classifier.h5')

In [1]:
import math
from collections import deque, Counter

vote_buffer = deque(maxlen=30) 

path = "/home/shin/deeplearning-repo-1/dataset/stand_shoulder_press/untitled.mp4"
cap = cv2.VideoCapture(0)
sequence_buffer = []

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arccos(np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0))
    return np.degrees(radians)

joint_angle_data = {}
for seq, label in zip(sequences, labels):
    if label not in joint_angle_data:
        joint_angle_data[label] = {"left_elbow": []}
    for frame in seq:
        landmarks = np.array(frame).reshape(-1, 4)  # (33, 4)
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][:2]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][:2]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value][:2]
        angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        joint_angle_data[label]["left_elbow"].append(angle)

target_pose = {}
for label, angles in joint_angle_data.items():
    target_pose[label] = {}
    for joint, values in angles.items():
        mean = np.mean(values)
        std = np.std(values)
        target_pose[label][joint] = (mean - 0.8 * std, mean + 0.8 * std)

def draw_correction(frame, angle, expected_range, joint_pos):
    min_angle, max_angle = expected_range
    if angle < min_angle:
        direction = "UP"
    elif angle > max_angle:
        direction = "DOWN"
    else:
        direction = None

    cv2.circle(frame, joint_pos, 10, (0, 0, 255), -1)
    if direction:
        if direction == "UP":
            cv2.arrowedLine(frame, joint_pos, (joint_pos[0], joint_pos[1]-50), (0, 0, 255), 3, tipLength=0.5)
        elif direction == "DOWN":
            cv2.arrowedLine(frame, joint_pos, (joint_pos[0], joint_pos[1]+50), (0, 0, 255), 3, tipLength=0.5)

cap = cv2.VideoCapture(0)
sequence_buffer = []
vote_buffer = deque(maxlen=30)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = []
        h, w, _ = frame.shape

        for lm in results.pose_landmarks.landmark:
            landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
        sequence_buffer.append(landmarks)

        if len(sequence_buffer) >= SEQUENCE_LENGTH:
            input_seq = np.expand_dims(sequence_buffer[-SEQUENCE_LENGTH:], axis=0)
            prediction = model.predict(input_seq)
            predicted_label = le.inverse_transform([np.argmax(prediction)])[0]
            vote_buffer.append(predicted_label)

            if len(vote_buffer) == vote_buffer.maxlen:
                most_common_label = Counter(vote_buffer).most_common(1)[0][0]
                cv2.putText(frame, f'Mode: {most_common_label}', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # 교정 가이드라인 (자동화된 기준값 사용)
                lm = results.pose_landmarks.landmark
                left_shoulder = (int(lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w),
                                 int(lm[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h))
                left_elbow = (int(lm[mp_pose.PoseLandmark.LEFT_ELBOW].x * w),
                              int(lm[mp_pose.PoseLandmark.LEFT_ELBOW].y * h))
                left_wrist = (int(lm[mp_pose.PoseLandmark.LEFT_WRIST].x * w),
                              int(lm[mp_pose.PoseLandmark.LEFT_WRIST].y * h))

                angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                expected_range = target_pose[most_common_label]["left_elbow"]
                draw_correction(frame, angle, expected_range, left_elbow)

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined